In [72]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [73]:
df = pd.read_csv("Bengaluru_House_Data.csv")

In [74]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [75]:
df.isna().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [76]:
def show_summary(dataframe):
    for w in dataframe.columns:
        print("Percent value Counts for {} is:".format(w.upper()),"\n")
        print (f'{round(((dataframe[w].value_counts()/dataframe[w].count())*100),2)}',"\n\n","----------------------------------------------------------------")

In [77]:
show_summary(df)

Percent value Counts for AREA_TYPE is: 

Super built-up  Area    65.99
Built-up  Area          18.15
Plot  Area              15.20
Carpet  Area             0.65
Name: area_type, dtype: float64 

 ----------------------------------------------------------------
Percent value Counts for AVAILABILITY is: 

Ready To Move    79.44
18-Dec            2.30
18-May            2.21
18-Apr            2.03
18-Aug            1.50
                 ...  
17-Jan            0.01
14-Nov            0.01
16-Nov            0.01
14-Jul            0.01
15-Dec            0.01
Name: availability, Length: 81, dtype: float64 

 ----------------------------------------------------------------
Percent value Counts for LOCATION is: 

Whitefield              4.05
Sarjapur  Road          3.00
Electronic City         2.27
Kanakpura Road          2.05
Thanisandra             1.76
                        ... 
Jaya Nagar East         0.01
Indira Nagar Stage 2    0.01
Binnamangala            0.01
Somanna Garden          0.

In [78]:
df.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')

In [79]:
df2 = df.drop(["area_type","availability","society","balcony"],axis = 1)

In [80]:
df2.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [81]:
df2.isna().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [82]:
df3= df2.dropna()
df3.isna().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [83]:

df3["bhk"] = df3["size"].apply(lambda x: int(x.split()[0]))




C:\Users\91988\anaconda3\envs\tensorflow_rk\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [84]:
df3.shape

(13246, 6)

In [85]:
# plt.figure(figsize=(15,5))
# sns.scatterplot(x= "bhk", y ="price", data = df1.sort_values(by="bhk"))
# plt.title("BHK v/S Price")
# plt.xticks(ticks= np.arange(0,50,2))

# plt.grid()
# plt.show()

In [86]:
df3["total_sqft"].unique()

# some values are in range and also the values are not float but str

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [87]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [88]:
df3[~df3["total_sqft"].apply(is_float)].head()

#these are the rows where the sqft values are in range 
# the sqft data is completely unstructured 

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2


In [89]:
def range_to_AvgSqft(x):
    tokens = x.split("-")
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [90]:
df4 = df3.copy()
df4["sqft_converted"] = df4["total_sqft"].apply(range_to_AvgSqft) 
df4 = df4.drop("total_sqft",axis=1)
df4 = df4[df4["sqft_converted"].notnull()]

In [91]:
df4.shape

(13200, 6)

In [92]:


# the sqft values in range are now covnerted to avg value of the range
# values that are not in range are converted to NaN

In [93]:
df5 = df4.copy()
df5["price_per_sqft"] = round(df5["price"]*100000/df5["sqft_converted"],1)

In [94]:
df5["price_per_sqft"].describe()

count    1.320000e+04
mean     7.920760e+03
std      1.067272e+05
min      2.678000e+02
25%      4.267725e+03
50%      5.438350e+03
75%      7.317100e+03
max      1.200000e+07
Name: price_per_sqft, dtype: float64

In [64]:
df5.to_csv("bhp.csv")

# Feature Engineering

In [104]:
df5["location"] = df5["location"].apply(lambda x:x.strip())
location_stats = df5["location"].value_counts().sort_values(ascending=False)

In [105]:
len(location_stats)

1287

In [106]:
len(location_stats[location_stats>10])

240

In [107]:
len(location_stats[location_stats<=10])

1047

In [108]:
location_stats_less_than_10 = location_stats[location_stats<=10]


In [109]:
len(location_stats[location_stats<=10])

1047

In [110]:
len(location_stats_less_than_10)

1047

In [111]:
df6 = df5.copy()
df6["location"] = df6["location"].apply(lambda x: "Others" if x in location_stats_less_than_10 else x)

In [112]:
len(df6["location"].unique())

241


# Outlier removal

In [120]:
# Assuming the standard size of one bedroom is approx 300sqft

len(df6[df6["sqft_converted"]/df6["bhk"]<300]

# these are anomalies 

744

In [118]:
#removing the outliers

df7 = df6[~(df6["sqft_converted"]/df6["bhk"]<300)]

In [119]:
df7.shape

(12456, 7)

In [124]:
df7["price_per_sqft"].describe()

count     12456.000000
mean       6308.503420
std        4168.127188
min         267.800000
25%        4210.500000
50%        5294.100000
75%        6916.700000
max      176470.600000
Name: price_per_sqft, dtype: float64

In [128]:
df7.groupby("location")["price_per_sqft"].agg("mean")

location
1st Block Jayanagar          12359.240000
1st Phase JP Nagar            9726.408696
2nd Phase Judicial Layout     4048.709091
2nd Stage Nagarbhavi         14381.277778
5th Block Hbr Layout          6622.100000
                                 ...     
Yelachenahalli                6363.376471
Yelahanka                     5204.895610
Yelahanka New Town            5860.657895
Yelenahalli                   4012.546154
Yeshwanthpur                  6620.547500
Name: price_per_sqft, Length: 241, dtype: float64

In [129]:
df7.groupby("location")["price_per_sqft"].agg("std")

location
1st Block Jayanagar          3931.937410
1st Phase JP Nagar           4802.123224
2nd Phase Judicial Layout     674.739542
2nd Stage Nagarbhavi         5272.432536
5th Block Hbr Layout         4675.407176
                                ...     
Yelachenahalli               2908.296524
Yelahanka                    1746.408779
Yelahanka New Town           3889.232044
Yelenahalli                   533.259359
Yeshwanthpur                 2028.994993
Name: price_per_sqft, Length: 241, dtype: float64

In [130]:
# try by joining the these columns to the table and comaparing abs

In [169]:
def remove_pps_outliers(df):
    df_out= pd.DataFrame()
    for key, subdf in df.groupby("location"):
        m = np.mean(subdf["price_per_sqft"])
        st = np.std(subdf["price_per_sqft"])
        reduced_df = subdf[ (subdf["price_per_sqft"]>(m-st)) & (subdf["price_per_sqft"]<=(m+st)) ]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out


df8 = remove_pps_outliers(df7)

In [170]:
# def trying(df):
#     df_out= pd.DataFrame()
#     for keys, subdf in df.groupby("location"):
#         print("The key is : \n",keys,"\n")
#         print ("The subdf is : \n",subdf[["location","size"]] ,"\n------------------------------\n")


In [171]:
# trying(df7)

In [172]:
df8.shape

(10242, 7)